In [19]:
import numpy as np
from enum import IntEnum


class Cards(IntEnum):
    DUKE = 0
    ASSA = 1
    CAPT = 2
    AMBA = 3
    CONT = 4
    
    def __str__(self):
        return self.name
    
class Actions(IntEnum):
    INCOME = 0
    FOREIGN_AID = 1
    COUP = 2
    TAX = 3
    ASSASSINATE = 4
    STEAL = 5
    EXCHANGE = 6
    
    def __str__(self):
        return self.name

class Counteractions(IntEnum):
    BLOCK_STEAL = 0
    BLOCK_ASSASSINATE = 1
    BLOCK_FOREIGN_AID = 2       

class Rules:
    STARTING_COINS = 2
    INCOME_AMT = 1
    FOREIGN_AID_AMT = 2
    TAX_AMT = 3
    STEAL_AMT = 2
    COUP_COST = 7
    ASSASSINATE_COST = 3
    
    
class Action_Response(IntEnum):
    PASS = 0
    CHALLENGE = 1
    COUNTERACT = 2
    
    def __str__(self):
        return self.name

necessary_card_for_action = {Actions.TAX : Cards.DUKE, Actions.ASSASSINATE: Cards.ASSA, Actions.STEAL: Cards.CAPT, Actions.EXCHANGE: Cards.AMBA}
necessary_card_for_counteraction = {Counteractions.BLOCK_FOREIGN_AID : [Cards.DUKE], Counteractions.BLOCK_ASSASSINATE: [Cards.CONT], Counteractions.BLOCK_STEAL: [Cards.CAPT,Cards.AMBA]}





DECK_LAYOUT = {
    Cards.DUKE : 3,
    Cards.ASSA : 3,
    Cards.CAPT : 3,
    Cards.AMBA : 3,
    Cards.CONT : 3
}

ALL_ACTIONS = list(Actions)

INCONTESTABLE_ACTIONS = [Actions.INCOME, Actions.COUP]






# class PlayerState:
#     def __init__(self,id):
#         self.id = id
#         self.coins = 0
#         self.cards = []
        
#     def give_card(self,card):
#         self.cards.append(card)
        
#     def __str__(self):
#         return f"Player {self.id}"
        
    

class CoupEnv:
    def __init__(self, num_players: int = 3, seed: int | None = None):
        
        self.rng = np.random.default_rng(seed)
        self.num_players = num_players
        self.deck = []
        self.reset()

    def reset(self):

        self.deck.clear()

        #supply the deck with cards according to their frequency
        for card,freq in DECK_LAYOUT.items():
            for _ in range(freq):
                self.deck.append(card)
        
        self.player_states = {}

        #dole each player two cards and two coins to start the game
        for i in range(self.num_players):

            card_1 = self._deal_card()
            card_2 = self._deal_card()
            
            self.player_states[i] = {
                "coins" : Rules.STARTING_COINS,
                "cards" : [card_1, card_2],
                "alive" : True
            }
   
    
    def apply_action(self, action, no_effect=False):
        #action should be {"player_id": id, "action_type" : int, "target_player_id": None}
        
        action_function = self._get_action_function(action)
        
        action_function(no_effect)

    
    def _get_action_function(self, action):

        action_type = action["action_type"]
        player_id = action["player_id"]
        target_player_id = action["target_player_id"]
        
        if action_type == Actions.INCOME:
            return lambda no_effect: self._mod_player_coins(player_id, change = Rules.INCOME_AMT)
            
        elif action_type == Actions.FOREIGN_AID:
            return lambda no_effect: self._mod_player_coins(player_id, change = Rules.FOREIGN_AID_AMT)

        elif action_type == Actions.COUP:
            def coup_function(no_effect):
                self._mod_player_coins(player_id, change = -Rules.COUP_COST)
                self.request_card_reveal(target_player_id)
            return coup_function
            
        elif action_type == Actions.TAX:
            return lambda no_effect: self._mod_player_coins(player_id, change = Rules.TAX_AMT)
            
        elif action_type == Actions.ASSASSINATE:
            def assassinate_function(no_effect):
                self._mod_player_coins(player_id, change = -Rules.ASSASSINATE_COST)
                self.request_card_reveal(target_player_id)
            return assassinate_function

        elif action_type == Actions.STEAL:
            def steal_function(no_effect):
                coins_to_steal = min(Rules.STEAL_AMT, self.player_states[target_player_id]["coins"])
                self._mod_player_coins(target_player_id, change = -coins_to_steal)
                self._mod_player_coins(player_id, change = coins_to_steal)
            return steal_function
            
        elif action_type == Actions.EXCHANGE:
            def exchange_function(no_effect):
                self.request_exchange(player_id)
            return exchange_function            
        
        else:
            raise Exception("invalid input")    
            
        
            #part of player state class
    def _mod_player_coins(self, player_id, change):
        if (self.player_states[player_id]["coins"] + change) < 0:
            raise Exception("action invalid- coins goes below 0")
        
        self.player_states[player_id]["coins"] += change
        # part of player state class
    def give_player_new_card(self, player_id):
        
        assert(len(self.player_states[player_id]["cards"]) == 1), "Player has too many cards or is out." 
        
        new_card = self._deal_card()

        self.player_states[player_id]["cards"].append(new_card)

    
    #part of env class is fine 
    def _get_legal_actions(self, player_id: int) -> list[int]:
        """Return list of legal actions for the given player, considering coins and board state."""
        state = self.player_states[player_id]

        if state["coins"] >= 10:
            return [Actions.COUP]

        actions = ALL_ACTIONS.copy()
        
        
        if state["coins"] < 7:
            actions.remove(Actions.COUP)
            
            if state["coins"] < 3:
                actions.remove(Actions.ASSASSINATE)
        
        return actions
    
    
    def get_targets(self, player_id:int) -> list[int]:
        return [id for id in self.player_states.keys() if id != player_id and self.player_states[id]["alive"]]

#part of deck class?
    def _deal_card(self)-> int:
        assert(self.deck and len(self.deck) > 0), "Deck Empty or Not Yet Initialized"

        card = Cards(self.rng.choice(self.deck))
        self.deck.remove(card)

        return card
        
        #part of player state class 
    def update_player_lives(self):
        for id,player in self.player_states.items():
            if (len(player["cards"]) == 0) and (player["alive"]):
                self.player_states[id]["alive"] = False
                print(f"player {id} died")
        

    def is_terminal(self) -> bool:

        #check that all but one player is dead
        one_player_alive = False
        
        for state in self.player_states.values():
            
            if one_player_alive and state['alive']:
                return False
            
            if state["alive"]:
                one_player_alive = True
        
        return True
            
            
        """Check if the game is over."""

    

In [38]:
class GameClient:
    
    MAX_ATTEMPTS = 10
    
    def __init__(self, is_cpu = True):
        self.is_cpu = is_cpu
        self.rng = None
        if self.is_cpu:
            self.rng = np.random.default_rng()
    
    
    
    def ask_input(self,options:list):
        return self.rng.choice(options)
        
            
   
        
        
        
    

In [43]:
def log(func):
        def wrapper(*args, **kwargs):
            print(f"[LOG] Calling {func.__name__} with args={args}, kwargs={kwargs}")
            result = func(*args, **kwargs)
            print(f"[LOG] {func.__name__} returned {result}")
            return result
        return wrapper
    
    
class GameController:
    
    def __init__(self, num_players=3):
        self.num_players = num_players
        self.env = CoupEnv(num_players)
        self.reset()
    
    def reset(self): 
        self.clients = []
        self.turn_order = [i for i in range(self.num_players)]
        self.game_over = False
        self.turn_num = 0
        self.clients = {player_id: GameClient() for player_id in range(self.num_players)}
        
        
    def game_loop(self):
        while not self.game_over:
            for player_id in self.turn_order:
                print(f"Turn number {self.turn_num}: player {player_id} go:" )
                self.take_turn(player_id)
                self.env.update_player_lives()
                

                if self.env.is_terminal():
                    self.game_over = True
                    break
                
            self.turn_num+=1
        
    def take_turn(self,player_id):
    
        
        print(f"player {player_id} is {"alive " if  self.env.player_states[player_id]["alive"] else "dead"}")
        
        if not self.env.player_states[player_id]["alive"]: return
        
        print(f"has {len(self.env.player_states[player_id]["cards"])} {self.env.player_states[player_id]["cards"]} and {self.env.player_states[player_id]["coins"]} coins")
        
        
        curr_player_action = self.request_player_action(player_id)


        curr_player_action_type = curr_player_action["action_type"]
        
        print(f"player {player_id} tries action: {curr_player_action_type}")

        if curr_player_action_type in INCONTESTABLE_ACTIONS:
            self.env.apply_action(curr_player_action)


            print("action uncontestable therefore passed")
            
            # next turn
            return
        

        # else action is contestable

        contest = self.propose_action(curr_player_action) #prompts all players for their response. The first non pass shorts the function and is returned
        #contest is a None, or a dict of {"type": in [challenge, counteract] , "from_player:int": player_id, "counteraction" : in [block foreign aid, block steal, block assassinate]}


        if not contest:
            self.env.apply_action(curr_player_action)


            print("action uncontested therefore passed")
            # next turn
            return

        contest_type = contest["type"]
        challenger = contest["from_player"]
        
        if contest_type == CHALLENGE:
            
            revealed_card = self.request_card_reveal(player_id) # p is prompted to reveal a card
            
            if revealed_card == necessary_card_for_action[curr_player_action_type]: #where necessary card is a dict mapping an action to the card needed for it -- will
                #p wins the challenge
                
                penalty_card = self.request_card_reveal(challenger) # challenger has to reveal a card if their challenge is wrong
                self.env.give_player_new_card(player_id) #p gets a new card
                self.env.deck.append(revealed_card) # the old card has to go back into the deck
                self.env.apply_action(curr_player_action) #action goes through anyway

                print("action challenged but challenge failed therefore passed")
            #else ie if revealed card shows p's bluff or they choose not to reveal their card
            # accuser wins challenge
            # nothing has to happen since request_card_reveal removed the card from the player p. We don't apply the action, just pass to the next turn
            print("action discarded")

            # next turn
            return
            
        #an action and counteraction have a 1:1 relationship, right? 
        if contest_type == COUNTERACT:

            counteraction = contest["counteraction"]
            counteract_challenge = self.propose_action(counteraction) #prompt all other players for a response to the counteraction
            
            if not counteract_challenge:
                self.env.apply_action(curr_player_action, no_effect = True) #the action is blocked so it should cost p money but have no effect

                print("action blocked because counteract was not challenged")
                # next turn
                return
                
            
            #else it is challenged ie. assassin from p, blocked by contessa from counteracter, someone says you don't have contessa 
            revealed_counteraction_card = self.request_card_reveal(challenger) # ask the counteractor to reveal a card
            
            if revealed_counteraction_card in necessary_card_for_counteraction[counteraction]:
                
                penalty_card = self.request_card_reveal(counteract_challenge["from_player"])
                self.env.give_player_new_card(challenger) #the successful counteractor gets a new card
                self.env.deck.append(revealed_counteraction_card)
                self.env.apply_action(curr_player_action, no_effect = True) 
                print("action blocked because counteract was challenge and passed")
                
                # next turn
                return

            #else the counteractor bluffed, they dont get a card back after revealing it
            
            self.env.apply_action(curr_player_action)
        


    
    @log
    def request_player_action(self, player_id):
        
        actions = self.env._get_legal_actions(player_id)
        
        action_type = self.clients[player_id].ask_input(actions)
        

        target_id = None
        if action_type in [Actions.COUP, Actions.STEAL, Actions.ASSASSINATE]:
            
            target_id = int(self.clients[player_id].ask_input(self.env.get_targets(player_id)))

        return {"player_id": player_id, "action_type": action_type, "target_player_id":target_id}


    @log
    def request_card_reveal(self, player_id):

       
        revealed_card = Cards(self.clients[player_id].ask_input(self.env.player_states[player_id]["cards"]))
        
        return revealed_card


    @log
    def request_exchange(self, player_id):

        player_cards = self.env.player_states[player_id]["cards"]
        num_player_cards = len(player_cards)
        
        #todo get user input
        first = self._deal_card()
        second = self._deal_card()
        
        show_these = [first, second] + player_cards

        user_new_card_1 = self.clients[player_id].ask_input(show_these)
        
        show_these.remove(user_new_card_1)
        
        user_new_card_2 = self.clients[player_id].ask_input(show_these)
        
        user_picked = [user_new_card_1, user_new_card_2]
        
        to_go_back = set(show_these) - set(user_picked)
        
        for card in to_go_back:
            self.env.deck.append(card)

        self.env.player_states[player_id]["cards"] = list(user_picked) 

        
    def propose_action(self, action) -> dict | None:
        # prompts all players for their response. The first non pass shorts the function and is returned
        # contest is a None, or a dict of {"type": in [challenge, counteract] , 
        # "from_player:int": player_id, "counteraction" : in [block foreign aid, block steal, block assassinate]}
        
        
        return None
    #end functions that require user input
    
    
        

In [44]:
gc = GameController()
gc.game_loop()

Turn number 0: player 0 go:
player 0 is alive 
has 2 [<Cards.DUKE: 0>, <Cards.CONT: 4>] and 2 coins
[LOG] Calling request_player_action with args=(<__main__.GameController object at 0x10aba7a10>, 0), kwargs={}
[LOG] request_player_action returned {'player_id': 0, 'action_type': np.int64(1), 'target_player_id': None}
player 0 tries action: 1
action uncontested therefore passed
Turn number 0: player 1 go:
player 1 is alive 
has 2 [<Cards.CAPT: 2>, <Cards.ASSA: 1>] and 2 coins
[LOG] Calling request_player_action with args=(<__main__.GameController object at 0x10aba7a10>, 1), kwargs={}
[LOG] request_player_action returned {'player_id': 1, 'action_type': np.int64(3), 'target_player_id': None}
player 1 tries action: 3
action uncontested therefore passed
Turn number 0: player 2 go:
player 2 is alive 
has 2 [<Cards.CAPT: 2>, <Cards.CONT: 4>] and 2 coins
[LOG] Calling request_player_action with args=(<__main__.GameController object at 0x10aba7a10>, 2), kwargs={}
[LOG] request_player_action retu

AttributeError: 'CoupEnv' object has no attribute 'request_exchange'